This is a performance testing with the dataset of adolescent on two model. A dual model setup is done here.

The first model Electra is going to train on the emotion and the intent of the user and the GPT-2 model is going to generate text based on the questions.

The datasets used here -> casual_lm_train.jsonl and masked_lm_train.jsonl

casual_lm_train is used for the GPT-2 training and the masked_lm train is for the Electra.

gpt-2 performance is going to evaluate by its perplexity that is how certain it's response is or other word what is it's accuracy to predict the next word.

electra's performance is evaluated by it's accuracy to the prediction of the emotion and intention of the questions.



In [2]:
pip install transformers datasets scikit-learn numpy pandas torch --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
from transformers import (
    ElectraTokenizerFast, ElectraForSequenceClassification,
    T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorWithPadding, DataCollatorForSeq2Seq
)
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support, confusion_matrix
import pandas as pd
import numpy as np
import random
import os
import json
from sklearn.model_selection import train_test_split


torch.manual_seed(42)

In [6]:
# data from JSONL files
def load_jsonl(file_path):
    data = []
    try:
        with open(file_path, 'r') as f:
            for line in f:
                data.append(json.loads(line.strip()))
        print(f"Loaded {len(data)} samples from {file_path}")
    except FileNotFoundError:
        print(f"Error: {file_path} not found!")
        return []
    return data

masked_lm_data = load_jsonl("/content/drive/MyDrive/Datasets/masked_lm_train.jsonl")
causal_lm_data = load_jsonl("/content/drive/MyDrive/Datasets/causal_lm_train.jsonl")

# convert for classification format for electra
extended_data = []

# masked LM data for emotion/intent classification
for item in masked_lm_data:
    sentence1 = item.get('sentence1', '')
    sentence2 = item.get('sentence2', '')

    if not sentence1:
        continue

    # extract emotion adn intent
    text_lower = sentence1.lower()

    # predict intent based on question patterns
    if any(word in text_lower for word in ['why', 'how', 'what', 'when', 'where', '?']):
        intent = "question"
    elif any(word in text_lower for word in ['help', 'support', 'advice']):
        intent = "seeking support"
    else:
        intent = "venting"

    # predict emotion based on content patterns - IMPROVED WITH BETTER DETECTION
    if any(word in text_lower for word in ['worthless', 'nobody likes', 'hate myself', 'useless', 'sad', 'depressed', 'down', 'upset', 'lonely', 'isolated', 'alone', 'feel like crying', 'feel awful', 'terrible', 'horrible', 'empty', 'hopeless']):
        emotion = "sad"
    elif any(word in text_lower for word in ['anxious', 'anxiety', 'worry', 'worried', 'stress', 'stressed', 'nervous', 'panic', 'overwhelmed', 'can\'t stop thinking', 'overthinking', 'scared', 'afraid', 'fear']):
        emotion = "anxious"
    elif any(word in text_lower for word in ['angry', 'mad', 'frustrated', 'annoyed', 'furious', 'hate', 'pissed', 'makes me angry', 'irritated', 'rage', 'livid']):
        emotion = "angry"
    elif any(word in text_lower for word in ['mother', 'father', 'parent', 'family', 'relationship', 'independence', 'autonomy', 'confused', 'don\'t understand', 'mixed up']):
        emotion = "confused"
    elif any(word in text_lower for word in ['growth', 'development', 'maturity', 'change', 'personal', 'curious', 'wonder', 'interested']):
        emotion = "curious"
    else:
        emotion = "thoughtful"

    if sentence2:
        response = sentence2
    else:
        if emotion == "sad":
            response = "I hear that you're going through a really difficult time right now. Those feelings are valid, and you're not alone. Would you like to talk about what's been making you feel this way?"
        elif emotion == "anxious":
            response = "It sounds like you're dealing with some anxiety, which can be really challenging. These feelings are completely normal. What specifically has been making you feel anxious?"
        elif emotion == "angry":
            response = "I can hear the frustration and anger in what you're sharing. These feelings are completely understandable. Sometimes it helps to talk about what's been making you feel this way."
        elif emotion == "confused":
            response = "That's a really important question about relationships and growing up. These feelings of confusion are normal during adolescence. Let's explore this together."
        elif emotion == "curious":
            response = "That's a great question about personal growth! It shows you're thinking deeply about your development. Let's discuss what this means for you."
        else:
            response = "Thank you for sharing that with me. I appreciate your thoughtfulness. Let's discuss this further."

    extended_data.append({
        "text": sentence1,
        "emotion": emotion,
        "intent": intent,
        "response": response
    })

for item in causal_lm_data:
    text = item.get('text', '')

    if not text:
        continue

    text_lower = text.lower()

    # predict intent and emotion for causal LM data
    if any(word in text_lower for word in ['?', 'why', 'how', 'what']):
        intent = "question"
    else:
        intent = "seeking support"

    if any(word in text_lower for word in ['autonomy', 'independence', 'mother', 'father']):
        emotion = "confused"
        response = "That's an important question about family relationships and independence. Let's explore this together."
    elif any(word in text_lower for word in ['growth', 'personal', 'development']):
        emotion = "curious"
        response = "Personal growth is a key part of adolescent development. It shows you're developing self-awareness."
    elif any(word in text_lower for word in ['purpose', 'life', 'meaning']):
        emotion = "thoughtful"
        response = "Having a sense of purpose is crucial for well-being. It gives direction and meaning to your experiences."
    else:
        emotion = "thoughtful"
        response = "That's a thoughtful question. Let's discuss this in more detail."

    extended_data.append({
        "text": text,
        "emotion": emotion,
        "intent": intent,
        "response": response
    })

# training examples for better understanding and emotion detection
additional_examples = [
    {
        "text": "I feel like nobody likes me at school and I'm always alone",
        "emotion": "sad",
        "intent": "venting",
        "response": "I'm really sorry you're feeling so alone at school. That sense of isolation can be really painful. Your feelings are completely valid, and I want you to know that you matter."
    },
    {
        "text": "Why do I always feel worthless and like I can't do anything right",
        "emotion": "sad",
        "intent": "question",
        "response": "Those feelings of worthlessness are really hard to carry. It's important to know that these thoughts don't reflect your true value as a person. You're asking a brave question by reaching out."
    },
    {
        "text": "How can I deal with my anxiety it's getting really bad",
        "emotion": "anxious",
        "intent": "seeking support",
        "response": "I'm glad you're reaching out about your anxiety. That takes courage. There are definitely strategies that can help manage anxiety. Let's work through some techniques together."
    },
    {
        "text": "I'm so angry at my parents they never listen to me",
        "emotion": "angry",
        "intent": "venting",
        "response": "It's really frustrating when you feel like your parents aren't hearing you. Those angry feelings make complete sense when you don't feel understood or respected."
    },
    {
        "text": "I can't stop overthinking everything and it's making me crazy",
        "emotion": "anxious",
        "intent": "venting",
        "response": "That constant overthinking sounds exhausting. When our minds race like that, it can feel overwhelming. You're not alone in experiencing this."
    },
    {
        "text": "Why do I feel so hopeless about everything",
        "emotion": "sad",
        "intent": "question",
        "response": "Feeling hopeless can be incredibly heavy and difficult to bear. These feelings are telling me that you're really struggling right now. What's been contributing to these feelings?"
    },
    {
        "text": "I feel empty inside like nothing matters anymore",
        "emotion": "sad",
        "intent": "venting",
        "response": "That emptiness you're describing sounds really painful. When everything feels meaningless, it can be so isolating. I'm here to listen and help you work through these feelings."
    },
    {
        "text": "I hate myself for being such a failure",
        "emotion": "sad",
        "intent": "venting",
        "response": "I hear so much pain in what you're sharing. Those harsh thoughts about yourself must be exhausting to carry. You're not a failure - you're a person going through a difficult time."
    },
    {
        "text": "Everything makes me worry and I can't relax",
        "emotion": "anxious",
        "intent": "venting",
        "response": "Living with that constant worry sounds really exhausting. When anxiety affects our ability to relax, it can feel overwhelming. Let's talk about what's been triggering these worries."
    }
]

extended_data.extend(additional_examples)

labels_emotion = list(set(d['emotion'] for d in extended_data))
labels_intent = list(set(d['intent'] for d in extended_data))

print(f"Total samples: {len(extended_data)}")
print(f"Emotion labels ({len(labels_emotion)}): {labels_emotion}")
print(f"Intent labels ({len(labels_intent)}): {labels_intent}")

for d in extended_data:
    d['emotion_label'] = labels_emotion.index(d['emotion'])
    d['intent_label'] = labels_intent.index(d['intent'])

# train-test split and dataset conversion
train_data, test_data = train_test_split(extended_data, test_size=0.15, random_state=42)

train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_data))

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

print("\nSample training data:")
for i in range(min(3, len(train_data))):
    sample = train_data[i]
    print(f"Text: {sample['text'][:80]}...")
    print(f"Emotion: {sample['emotion']}, Intent: {sample['intent']}")
    print(f"Response: {sample['response'][:80]}...")
    print("-" * 40)

Loaded 600 samples from /content/drive/MyDrive/Datasets/masked_lm_train.jsonl
Loaded 600 samples from /content/drive/MyDrive/Datasets/causal_lm_train.jsonl
Total samples: 1209
Emotion labels (6): ['anxious', 'curious', 'confused', 'thoughtful', 'sad', 'angry']
Intent labels (3): ['seeking support', 'question', 'venting']
Training samples: 1027
Test samples: 182

Sample training data:
Text: How does your relationship with your mother affect your autonomy?...
Emotion: confused, Intent: question
Response: That's an important question about family relationships and independence. Let's ...
----------------------------------------
Text: Why is purpose in life important for teenagers?...
Emotion: thoughtful, Intent: question
Response: Having a sense of purpose is crucial for well-being. It gives direction and mean...
----------------------------------------
Text: What does a high personal growth score mean during adolescence?...
Emotion: curious, Intent: question
Response: Personal growth is 

In [7]:
# electra classifier for emotion and intent prediction

classifier_model = ElectraForSequenceClassification.from_pretrained(
    "google/electra-small-discriminator",
    num_labels=len(labels_emotion)
)
classifier_tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-small-discriminator")

def tokenize_and_encode(batch):
    return classifier_tokenizer(batch['text'], truncation=True, padding=True, max_length=128)

# tokenization of datasets
train_tokenized = train_dataset.map(tokenize_and_encode, batched=True)
test_tokenized = test_dataset.map(tokenize_and_encode, batched=True)

train_tokenized = train_tokenized.rename_column("emotion_label", "labels")
test_tokenized = test_tokenized.rename_column("emotion_label", "labels")

train_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# training arguments
classifier_args = TrainingArguments(
    output_dir="./classifier_output",
    logging_steps=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    seed=42,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

# trainer
trainer = Trainer(
    model=classifier_model,
    args=classifier_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=classifier_tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=classifier_tokenizer),
    compute_metrics=lambda p: {
        "accuracy": accuracy_score(
            p.label_ids, np.argmax(p.predictions, axis=1)
        )
    }
)

print("Training ELECTRA classifier...")
trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

classifier_model.save_pretrained("./electra_emotion_classifier")
classifier_tokenizer.save_pretrained("./electra_emotion_classifier")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

/tmp/ipython-input-3845594068.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training ELECTRA classifier...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.107700,0.056609,0.989011
2,0.007200,0.022375,0.994505
3,0.004000,0.017883,0.994505
4,0.002900,0.016544,0.994505
5,0.002500,0.016114,0.994505


Evaluation results: {'eval_loss': 0.016114117577672005, 'eval_accuracy': 0.9945054945054945, 'eval_runtime': 0.5733, 'eval_samples_per_second': 317.483, 'eval_steps_per_second': 80.243, 'epoch': 5.0}


('./electra_emotion_classifier/tokenizer_config.json',
 './electra_emotion_classifier/special_tokens_map.json',
 './electra_emotion_classifier/vocab.txt',
 './electra_emotion_classifier/added_tokens.json',
 './electra_emotion_classifier/tokenizer.json')

In [10]:
# training for T5
print("Preparing T5 training data...")

prompted_samples = []

for d in extended_data:
    prompted_samples.append({
        "input_text": f"generate response: emotion: {d['emotion']}, intent: {d['intent']}\nUser: {d['text']}",
        "target_text": d['response']
    })

print(f"Total T5 training samples: {len(prompted_samples)}")

# model loading
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

def encode_t5(example):
    # Encode the input text for the encoder
    tokenized_input = t5_tokenizer(
        example['input_text'],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )

    # Encode the target text for the decoder
    tokenized_target = t5_tokenizer(
        example['target_text'],
        truncation=True,
        padding="max_length",
        max_length=64,
        return_tensors="pt"
    )

    labels = tokenized_target.input_ids
    labels[labels == t5_tokenizer.pad_token_id] = -100

    return {
        "input_ids": tokenized_input.input_ids.squeeze(0),
        "attention_mask": tokenized_input.attention_mask.squeeze(0),
        "labels": labels.squeeze(0)
    }

# dataset to train and split
t5_train_data, t5_test_data = train_test_split(prompted_samples, test_size=0.1, random_state=42)

t5_train_dataset = Dataset.from_list(t5_train_data)
t5_test_dataset = Dataset.from_list(t5_test_data)

print(f"T5 training samples: {len(t5_train_dataset)}")
print(f"T5 test samples: {len(t5_test_dataset)}")

t5_train_dataset = t5_train_dataset.map(encode_t5)
t5_test_dataset = t5_test_dataset.map(encode_t5)

t5_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
t5_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# training arguments for T5
t5_args = TrainingArguments(
    output_dir="./t5_response_generator",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=3e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
)

# trainer
t5_trainer = Trainer(
    model=t5_model,
    args=t5_args,
    train_dataset=t5_train_dataset,
    eval_dataset=t5_test_dataset,
    tokenizer=t5_tokenizer,
    data_collator=DataCollatorForSeq2Seq(t5_tokenizer, model=t5_model)
)

print("Training T5 response generator...")
t5_trainer.train()
t5_eval_results = t5_trainer.evaluate()
print(f"T5 evaluation results: {t5_eval_results}")

# Calculate perplexity for better evaluation of generative model
import math
eval_loss = t5_eval_results['eval_loss']
perplexity = math.exp(eval_loss)
print(f"T5 Perplexity: {perplexity:.2f}")
print(f"T5 Evaluation Loss: {eval_loss:.4f}")

# Save the model
t5_model.save_pretrained("./t5_adolescent_chatbot_final")
t5_tokenizer.save_pretrained("./t5_adolescent_chatbot_final")

Preparing T5 training data...
Total T5 training samples: 1209


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,1.704300,1.226831
2,0.549000,0.259585
3,0.228700,0.196201
4,0.213700,0.193729
5,0.177500,0.192673


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


T5 evaluation results: {'eval_loss': 0.1926727294921875, 'eval_runtime': 1.3137, 'eval_samples_per_second': 92.107, 'eval_steps_per_second': 46.434, 'epoch': 5.0}
T5 Perplexity: 1.21
T5 Evaluation Loss: 0.1927


('./t5_adolescent_chatbot_final/tokenizer_config.json',
 './t5_adolescent_chatbot_final/special_tokens_map.json',
 './t5_adolescent_chatbot_final/spiece.model',
 './t5_adolescent_chatbot_final/added_tokens.json')

In [20]:
# Inference Functions for Two-Model Chatbot Architecture

def classify_emotion_intent(text):

    # Electra model loading to the device
    device = next(classifier_model.parameters()).device

    inputs = classifier_tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    classifier_model.eval()
    with torch.no_grad():
        outputs = classifier_model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
        confidence = torch.softmax(outputs.logits, dim=1).max().item()

    return {
        "emotion": labels_emotion[pred],
        "confidence": confidence,
        "all_scores": {labels_emotion[i]: score.item() for i, score in enumerate(torch.softmax(outputs.logits, dim=1)[0])}
    }

def generate_response(text, max_length=150, temperature=0.9):
    # Classification for emotion and intent
    classification = classify_emotion_intent(text)
    emotion = classification["emotion"]

    # Determine intent from the text using rule-based approach (as Electra is only for emotion)
    text_lower = text.lower()
    if any(word in text_lower for word in ['why', 'how', 'what', 'when', 'where', '?']):
        intent = "question"
    elif any(word in text_lower for word in ['help', 'support', 'advice']):
        intent = "seeking support"
    else:
        intent = "venting"

    # Create a prompt for T5, matching the training format
    prompt = f"generate response: emotion: {emotion}, intent: {intent}\nUser: {text}"

    device = next(t5_model.parameters()).device # Use t5_model

    # Generate response with T5
    input_ids = t5_tokenizer(prompt, return_tensors="pt").input_ids.to(device) # Use t5_tokenizer
    t5_model.eval() # Use t5_model

    with torch.no_grad():
        output = t5_model.generate( # Use t5_model
            input_ids,
            max_new_tokens=max_length, # Increased max_new_tokens for longer responses
            temperature=temperature, # Adjusted temperature for more creativity
            do_sample=True,
            top_p=0.9,
            top_k=50,
            pad_token_id=t5_tokenizer.pad_token_id, # Use t5_tokenizer
            eos_token_id=t5_tokenizer.eos_token_id, # Use t5_tokenizer
            repetition_penalty=1.2,
            no_repeat_ngram_size=4 # Increased to 4 to discourage repetition of n-grams
        )

    # Decode the generated tokens. For T5, this output directly contains the response.
    # The generate method for T5 returns only the generated part, not including the input prompt.
    assistant_response = t5_tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # If response is too short, generic, or largely a repetition of the input, provide a fallback
    user_input_lower = text.lower().strip()
    response_lower = assistant_response.lower().strip()

    # Check for short, empty, or direct repetition/near repetition
    if (len(assistant_response) < 10 or
       response_lower in [user_input_lower, ""] or
       response_lower.startswith(user_input_lower) or
       (len(user_input_lower.split()) > 3 and response_lower.startswith(" ".join(user_input_lower.split()[:3]))) or
       (len(response_lower.split()) < len(user_input_lower.split()) and user_input_lower.startswith(response_lower))):
        if emotion == "sad":
            assistant_response = "It sounds like you're going through a really tough time, and those feelings of sadness are completely valid. I'm here to listen without judgment. Can you tell me more about what's been weighing on your mind lately?"
        elif emotion == "anxious":
            assistant_response = "It sounds like you're grappling with a lot of anxiety, which can be incredibly overwhelming. Please know that it's okay to feel this way. Perhaps we could explore what specific thoughts or situations are making you feel anxious?"
        elif emotion == "angry":
            assistant_response = "I can definitely sense your frustration and anger. Those feelings are absolutely understandable, especially when things feel unfair or misunderstood. What's been making you feel this way?"
        elif emotion == "confused":
            assistant_response = "That's a very insightful question, and it's completely normal to feel confused about certain things, especially during adolescence. Can you elaborate a bit more on what aspects are making you feel uncertain?"
        elif emotion == "curious":
            assistant_response = "That's an excellent question to be curious about! It shows you're thinking deeply. What aspects of this topic are you most interested in exploring further?"
        else:
            assistant_response = "Thank you for sharing that with me. I'm here to support you. Could you tell me a bit more about what you're experiencing or what's on your mind?"

    return {
        "response": assistant_response,
        "emotion_detected": emotion,
        "confidence": classification["confidence"],
        "emotion_scores": classification["all_scores"]
    }

# Testing
print("Testing Two-Model Chatbot Architecture")
print("=" * 50)

test_inputs = [
    "I feel like nobody likes me at school.",
    "Why do I always feel worthless?",
    "How can I deal with my anxiety?",
    "I'm angry at my parents for not understanding me.",
    "What does personal growth mean for teenagers?",
    "How does my relationship with my mother affect my independence?"
]

for i, user_input in enumerate(test_inputs, 1):
    print(f"\n Test {i}: {user_input}")
    print("-" * 30)

    result = generate_response(user_input)

    print(f"Emotion Detected: {result['emotion_detected']} (confidence: {result['confidence']:.2f})")
    print(f"Assistant Response: {result['response']}")

    print(f"\nEmotion Analysis:")
    for emotion, score in result['emotion_scores'].items():
        print(f"   {emotion}: {score:.3f}")

print("\n" + "=" * 50)
print("Model Evaluation on Test Set")
print("=" * 50)

correct_predictions = 0
total_predictions = 0

for item in test_data[:10]: # Using a subset for quick evaluation
    prediction = classify_emotion_intent(item['text'])
    actual_emotion = item['emotion']
    predicted_emotion = prediction['emotion']

    if predicted_emotion == actual_emotion:
        correct_predictions += 1
    total_predictions += 1

    print(f"Text: {item['text'][:50]}...")
    print(f"Actual: {actual_emotion} | Predicted: {predicted_emotion} | {'Correct' if predicted_emotion == actual_emotion else 'Wrong'}")
    print()

accuracy = correct_predictions / total_predictions
print(f"Classification Accuracy: {accuracy:.2f} ({correct_predictions}/{total_predictions})")

Testing Two-Model Chatbot Architecture

 Test 1: I feel like nobody likes me at school.
------------------------------
Emotion Detected: sad (confidence: 0.30)
Assistant Response: It sounds like you're going through a really tough time, and those feelings of sadness are completely valid. I'm here to listen without judgment. Can you tell me more about what's been weighing on your mind lately?

Emotion Analysis:
   anxious: 0.232
   curious: 0.073
   confused: 0.085
   thoughtful: 0.074
   sad: 0.304
   angry: 0.232

 Test 2: Why do I always feel worthless?
------------------------------
Emotion Detected: sad (confidence: 0.30)
Assistant Response: That's an important question about self-worth. Let's explore this together.

Emotion Analysis:
   anxious: 0.228
   curious: 0.074
   confused: 0.088
   thoughtful: 0.077
   sad: 0.302
   angry: 0.230

 Test 3: How can I deal with my anxiety?
------------------------------
Emotion Detected: sad (confidence: 0.27)
Assistant Response: I'm sure th

In [22]:
# Interactive Testing Cell - Test Your Own Questions
print("Test the chatbot with your own questions on different topics!")
print("Type 'quit' to exit")
print("-" * 50)

def interactive_chat():
    while True:
        user_input = input("\nYour question: ")

        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("Thanks for testing! Goodbye!")
            break

        if not user_input.strip():
            print("Please enter a question.")
            continue

        print(f"\nProcessing: '{user_input}'")
        print("-" * 40)

        try:
            # Get the response from our chatbot
            result = generate_response(user_input)

            print(f"Emotion Detected: {result['emotion_detected']}")
            print(f"Assistant Response: {result['response']}")

            # Removed detailed emotion analysis for a cleaner chat experience
            # If you want to see detailed emotion analysis again, uncomment the lines below:
            # print(f"\nDetailed Emotion Analysis:")
            # sorted_emotions = sorted(result['emotion_scores'].items(), key=lambda x: x[1], reverse=True)
            # for emotion, score in sorted_emotions:
            #     bar = "||" * int(score * 20)  # Visual bar
            #     print(f"   {emotion:>10}: {score:.3f} {bar}")

        except Exception as e:
            print(f"Error occurred: {str(e)}")
            print("Please try a different question.")

# Test with some diverse questions first
test_questions = [
    "I'm having trouble sleeping because I keep thinking about my future",
    "My friends seem to be moving on without me and I feel left behind",
    "I don't know what I want to do with my life and it scares me",
    "I feel pressure to be perfect all the time",
    "I'm struggling with my identity and who I really am",
    "I feel like I'm not good enough compared to others on social media",
    "I'm worried about climate change and the future of the world",
    "I feel like my teachers don't understand me",
    "I'm having conflicts with my siblings",
    "I feel overwhelmed by all the expectations people have of me"
]

print("\nTesting with diverse questions first:")
print("=" * 50)

for i, question in enumerate(test_questions, 1):
    print(f"\nTest {i}: {question}")
    print("-" * 30)

    try:
        result = generate_response(question)
        print(f"Emotion: {result['emotion_detected']} ({result['confidence']:.2f})")
        print(f"Response: {result['response']}")

        # Check if response is too generic/repetitive
        generic_phrases = [
            "That's true! In fact",
            "the most common response",
            "not really that into academics"
        ]

        is_generic = any(phrase in result['response'] for phrase in generic_phrases)
        if is_generic:
            print("WARNING: Response appears to be generic/repetitive")

    except Exception as e:
        print(f"Error: {str(e)}")

print(f"\n{'='*50}")
print("Now you can test with your own questions!")
print("Try asking about topics like:")
print("- Relationships and friendships")
print("- School and academic pressure")
print("- Future career concerns")
print("- Social media and self-image")
print("- Family dynamics")
print("- Personal growth and identity")
print("- Mental health and coping")
print(f"{'='*50}")

interactive_chat()

Test the chatbot with your own questions on different topics!
Type 'quit' to exit
--------------------------------------------------

Testing with diverse questions first:

Test 1: I'm having trouble sleeping because I keep thinking about my future
------------------------------
Emotion: sad (0.30)
Response: It sounds like you're going through a really tough time, and those feelings of sadness are completely valid. I'm here to listen without judgment. Can you tell me more about what's been weighing on your mind lately?

Test 2: My friends seem to be moving on without me and I feel left behind
------------------------------
Emotion: sad (0.30)
Response: I feel left behind, leaving me behind, and moving on without me. This has been linked to my facebook friends' move-on without me.

Test 3: I don't know what I want to do with my life and it scares me
------------------------------
Emotion: sad (0.29)
Response: It sounds like you're going through a really tough time, and those feelings of